In [19]:
import plotly.express as px
import pandas as pd


In [20]:
# data as stacked bar charts
data = {
    "Organism": [],
    "Read Count": [],
    "Sample": []
}
samples = []
min_reads = 0
total_others = []

with open('kraken2.csv', 'r') as f:
    raw_data = f.read().split("\n")
    
    samples = ["Sample "+x for x in raw_data[0].split(",") if x != ""]
    for s in samples:
        total_others.append(0)
    
    for r in raw_data[1:]:
        do_add = False
        tmp_new = [0 for s in samples]
        
        tmp_r = r.split(",")
        if len(tmp_r) > 1:
            org = "|".join([x for x in tmp_r[:-len(samples)] if x != "-"])

            for i in range(1, len(samples)):
                tmp_new[i] = tmp_r[-i]
                
                if tmp_new[i] != "NaN" and int(tmp_new[i]) >= min_reads:
                    do_add = True
                
            if do_add:
                for i in range(1, len(samples)):
                    data["Organism"].append(org)
                    data["Read Count"].append(tmp_new[i])
                    data["Sample"].append(samples[i])
            else:
                for i in range(1, len(samples)):
                    if tmp_new[i] != "NaN":
                        total_others[i] += int(tmp_new[i])
    
for i in range(1, len(samples)):
    data["Organism"].append("other")
    data["Read Count"].append(total_others[i])
    data["Sample"].append(samples[i])
    
df = pd.DataFrame.from_dict(data)
df["Read Count"] = df["Read Count"].astype(float)


In [21]:
fig = px.bar(df, x="Sample", y="Read Count", color='Organism')
fig.update_layout(showlegend=False)
fig.write_html("kraken2.html")
fig.show()
